# Final Caluclator Learning module and translator

In [42]:
import os
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import string
import mediapipe as mp
import difflib
import spacy
from tkinter import font as tkFont
import speech_recognition as sr
import threading
from queue import Queue
import time
import re
from collections import defaultdict

In [43]:

# MediaPipe and Dataset Setup
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

DATASET_PATH = r"D:\ROHIT\1400+data"
IMAGE_DATASET_PATH = r"D:\ROHIT\Images_path"
nlp = spacy.load("en_core_web_sm")

# Stop Words and Speech Recognition Queue
STOP_WORDS = {
    'a', 'an', 'the', 'was', 'if', 'for', 'and', 'or', 'but', 
    'with', 'are', 'is', 'in', 'on', 'at', 'to', 'of', 'by', 
    'as', 'it', 'this', 'that', 'these', 'those', '?', '!'
}

NEGATIONS = {"not", "no", "never", "none", "n't"}
QUESTION_WORDS = {'who', 'what', 'where', 'when', 'why', 'how', 'which', 'whom', 'whose'}

speech_queue = Queue()


In [44]:

# Calculator operations mapping
CALCULATOR_SYMBOLS = {
    '+': 'addition',
    '-': 'subtract',
    '*': 'multiply',
    '/': 'divide',
    '=': 'equal',
    '.': 'dot'
}

# Color Scheme
COLORS = {
    "dark_bg": "#121212",
    "darker_bg": "#0A0A0A",
    "light_bg": "#1E1E1E",
    "lighter_bg": "#2E2E2E",
    "primary": "#BB86FC",
    "primary_variant": "#3700B3",
    "secondary": "#03DAC6",
    "error": "#CF6679",
    "text_primary": "#FFFFFF",
    "text_secondary": "#B3B3B3",
    "button_bg": "#333333",
    "button_active": "#424242",
    "pose_color": (0, 255, 0),
    "left_hand_color": (255, 0, 0),
    "right_hand_color": (255, 0, 255),
    "face_color": (255, 69, 126),
    "connection_color": (255, 255, 0),
    "neck_color": (0, 255, 255),
    "background_color": (0, 0, 0),
    "calc_display": "#1E1E1E",
    "calc_button_num": "#333333",
    "calc_button_op": "#03DAC6",
    "calc_button_eq": "#BB86FC",
    "calc_button_clear": "#CF6679"
}


In [45]:

def build_word_variations_map():
    variations_map = {
        'you': {'your', 'yours', "you're", "you've", "you'll"},
        'my': {'me', 'my', 'i am ', "i'm", 'myself', 'mine'},
        'mother': {'She', 'her'},
        'father': {'He', 'him'},
        'name': {'names', 'named'},
        'what': {"what's", "what're"},
        'eat': {'ate', 'eating', 'eaten'},
        'run': {'ran', 'running'},
    }
    
    variations_map_copy = defaultdict(set)
    for root, variations in variations_map.items():
        variations_map_copy[root] = variations.copy()
        if root.endswith(('s', 'x', 'z', 'ch', 'sh')):
            plural = root + 'es'
        else:
            plural = root + 's'
        variations_map_copy[root].add(plural)
    
    return dict(variations_map_copy)

WORD_VARIATIONS_MAP = build_word_variations_map()

def split_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]


## Gloss model 

In [46]:

def convert_to_isl(english_sentence):
    doc = nlp(english_sentence.lower())
    
    subject = []
    obj = []
    verb = []
    adjectives = []
    question_word = []
    negation = []
    
    for token in doc:
        word = token.text
        if word in STOP_WORDS or token.is_punct:
            continue
            
        if word in QUESTION_WORDS:
            question_word.append(word.upper())
            continue
            
        if word in NEGATIONS:
            negation.append(word.upper())
            continue
            
        if "subj" in token.dep_:
            subject.append(word.upper())
            continue
            
        if "obj" in token.dep_:
            obj.append(word.upper())
            continue
            
        if token.pos_ == "VERB":
            verb.append(token.lemma_.upper())
            continue
            
        if token.pos_ == "ADJ":
            adjectives.append(word.upper())
            continue
            
        obj.append(word.upper())
    
    isl_gloss = subject + obj + verb + adjectives + question_word + negation
    return " ".join(isl_gloss)


In [47]:

def process_multiple_sentences(text):
    sentences = split_sentences(text)
    return [convert_to_isl(sent) for sent in sentences]

def get_valid_classes(input_text):
    if not os.path.exists(DATASET_PATH):
        return []
    
    isl_sentences = process_multiple_sentences(input_text)
    available_classes = os.listdir(DATASET_PATH)
    available_classes_lower = {label.lower().strip(): label for label in available_classes}
    
    matched_classes = []
    
    for isl_text in isl_sentences:
        input_text = isl_text.strip().lower()
        
        if not input_text or input_text in STOP_WORDS:
            continue
        
        if input_text in available_classes_lower:
            matched_classes.append(available_classes_lower[input_text])
            continue
        
        words = input_text.split()
        for phrase_length in range(4, 1, -1):
            for i in range(len(words) - phrase_length + 1):
                phrase = ' '.join(words[i:i+phrase_length])
                if phrase in available_classes_lower:
                    before_phrase = ' '.join(words[:i])
                    after_phrase = ' '.join(words[i+phrase_length:])
                    
                    result = []
                    if before_phrase:
                        result.extend(get_valid_classes(before_phrase))
                    result.append(available_classes_lower[phrase])
                    if after_phrase:
                        result.extend(get_valid_classes(after_phrase))
                    
                    matched_classes.extend(result)
                    break
            else:
                continue
            break
        else:
            components = []
            current_word = ""
            for char in input_text:
                if char.isalpha() or char.isdigit():
                    current_word += char
                else:
                    if current_word:
                        components.append(current_word)
                        current_word = ""
                    if char.strip():
                        components.append(char)
            if current_word:
                components.append(current_word)
            
            for component in components:
                if not component.strip() or component.lower() in STOP_WORDS:
                    continue
                    
                if component.isdigit():
                    for digit in component:
                        if digit.lower() in available_classes_lower:
                            matched_classes.append(available_classes_lower[digit.lower()])
                    continue
                    
                if len(component) == 1 and component.isalpha():
                    if component.lower() in available_classes_lower:
                        matched_classes.append(available_classes_lower[component.lower()])
                    continue
                    
                if len(component) > 1 and component.isalpha():
                    lower_component = component.lower()
                    
                    if lower_component in available_classes_lower:
                        matched_classes.append(available_classes_lower[lower_component])
                        continue
                        
                    doc = nlp(lower_component)
                    lemma = doc[0].lemma_ if doc else lower_component
                    
                    if lemma in available_classes_lower:
                        matched_classes.append(available_classes_lower[lemma])
                        continue
                        
                    found_variation = False
                    for root_word, variations in WORD_VARIATIONS_MAP.items():
                        if lower_component in variations and root_word in available_classes_lower:
                            matched_classes.append(available_classes_lower[root_word])
                            found_variation = True
                            break
                            
                    if found_variation:
                        continue
                        
                    for letter in component.upper():
                        letter_lower = letter.lower()
                        if letter_lower in available_classes_lower:
                            matched_classes.append(available_classes_lower[letter_lower])
                    continue
                    
                lower_component = component.lower()
                if lower_component in available_classes_lower:
                    matched_classes.append(available_classes_lower[lower_component])
                    continue
                    
                doc = nlp(component.lower())
                lemmatized_words = [token.lemma_ for token in doc if token.text not in STOP_WORDS]
                
                for word in lemmatized_words:
                    if word in available_classes_lower:
                        matched_classes.append(available_classes_lower[word])
                        continue
                        
                    for root_word, variations in WORD_VARIATIONS_MAP.items():
                        if word in variations and root_word in available_classes_lower:
                            matched_classes.append(available_classes_lower[root_word])
                            continue
                            
                    closest_match = difflib.get_close_matches(word, available_classes_lower.keys(), n=1, cutoff=0.6)
                    if closest_match:
                        matched_classes.append(available_classes_lower[closest_match[0]])
    
    return matched_classes


## Image Translation model 

In [48]:

def clear_image_display():
    """Clear the image display completely"""
    if hasattr(root, 'word_image_display'):
        root.word_image_display.config(image=None)
        root.word_image_label.config(text="")

def display_image_for_word(word):
    """Display image only if it exists in the dataset"""
    # First clear any existing image
    clear_image_display()
    
    if not os.path.exists(IMAGE_DATASET_PATH):
        return False
    
    clean_word = word.strip().lower()
    clean_word = ''.join([c for c in clean_word if c.isalpha() or c.isdigit()])
    
    if not clean_word:
        return False
    
    # Only show images for single words
    if len(clean_word.split()) > 1:
        return False
    
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']
    image_path = None
    
    for ext in image_extensions:
        potential_path = os.path.join(IMAGE_DATASET_PATH, f"{clean_word}{ext}")
        if os.path.exists(potential_path):
            image_path = potential_path
            break
    
    if not image_path:
        return False
    
    try:
        img = Image.open(image_path)
        img = img.resize((300, 300), Image.LANCZOS)
        
        if not hasattr(root, 'word_image_label'):
            root.word_image_label = ttk.Label(image_frame, text="Image:", style="TLabel")
            root.word_image_label.pack(pady=(0, 5))
            
            root.word_image_display = ttk.Label(image_frame)
            root.word_image_display.pack(pady=(0, 10))
        
        imgtk = ImageTk.PhotoImage(image=img)
        root.word_image_display.imgtk = imgtk
        root.word_image_display.config(image=imgtk)
        root.word_image_label.config(text=f"Image for '{clean_word}':")
        return True
    except Exception as e:
        print(f"Error loading image: {e}")
        clear_image_display()
        return False

In [49]:


def draw_colored_landmarks(image, results):
    image_height, image_width, _ = image.shape
    
    if results.pose_landmarks:
        POSE_CONNECTIONS_TO_KEEP = [
            (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_SHOULDER),
            (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.LEFT_ELBOW),
            (mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.LEFT_WRIST),
            (mp_holistic.PoseLandmark.RIGHT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_ELBOW),
            (mp_holistic.PoseLandmark.RIGHT_ELBOW, mp_holistic.PoseLandmark.RIGHT_WRIST),
            (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.LEFT_HIP),
            (mp_holistic.PoseLandmark.RIGHT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_HIP),
            (mp_holistic.PoseLandmark.LEFT_HIP, mp_holistic.PoseLandmark.RIGHT_HIP),
            (mp_holistic.PoseLandmark.LEFT_HIP, mp_holistic.PoseLandmark.LEFT_KNEE),
            (mp_holistic.PoseLandmark.LEFT_KNEE, mp_holistic.PoseLandmark.LEFT_ANKLE),
            (mp_holistic.PoseLandmark.RIGHT_HIP, mp_holistic.PoseLandmark.RIGHT_KNEE),
            (mp_holistic.PoseLandmark.RIGHT_KNEE, mp_holistic.PoseLandmark.RIGHT_ANKLE)
        ]
        
        for connection in POSE_CONNECTIONS_TO_KEEP:
            start_idx = connection[0].value
            end_idx = connection[1].value
            
            start_point = results.pose_landmarks.landmark[start_idx]
            end_point = results.pose_landmarks.landmark[end_idx]
            
            start_x = int(start_point.x * image_width)
            start_y = int(start_point.y * image_height)
            end_x = int(end_point.x * image_width)
            end_y = int(end_point.y * image_height)
            
            cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["pose_color"], 10)
        
        if results.face_landmarks:
            left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value]
            chin = results.face_landmarks.landmark[152]
            
            neck_x = int(((left_shoulder.x + right_shoulder.x) / 2) * image_width)
            neck_y = int(((left_shoulder.y + right_shoulder.y) / 2) * image_height)
            
            chin_x = int(chin.x * image_width)
            chin_y = int(chin.y * image_height)
            
            cv2.line(image, (neck_x, neck_y), (chin_x, chin_y), COLORS["neck_color"], 10)
    
    if results.left_hand_landmarks:
        for connection in mp_holistic.HAND_CONNECTIONS:
            start_idx = connection[0]
            end_idx = connection[1]
            
            start_point = results.left_hand_landmarks.landmark[start_idx]
            end_point = results.left_hand_landmarks.landmark[end_idx]
            
            start_x = int(start_point.x * image_width)
            start_y = int(start_point.y * image_height)
            end_x = int(end_point.x * image_width)
            end_y = int(end_point.y * image_height)
            
            cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["left_hand_color"], 4)
    
    if results.right_hand_landmarks:
        for connection in mp_holistic.HAND_CONNECTIONS:
            start_idx = connection[0]
            end_idx = connection[1]
            
            start_point = results.right_hand_landmarks.landmark[start_idx]
            end_point = results.right_hand_landmarks.landmark[end_idx]
            
            start_x = int(start_point.x * image_width)
            start_y = int(start_point.y * image_height)
            end_x = int(end_point.x * image_width)
            end_y = int(end_point.y * image_height)
            
            cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["right_hand_color"], 4)
    
    if results.face_landmarks:
        outline_connections = [
            (10, 338), (338, 297), (297, 332), (332, 284), (284, 251), 
            (251, 389), (389, 356), (356, 454), (454, 323), (323, 361), 
            (361, 288), (288, 397), (397, 365), (365, 379), (379, 378), 
            (378, 400), (400, 377), (377, 152), (152, 148), (148, 176), 
            (176, 149), (149, 150), (150, 136), (136, 172), (172, 58), 
            (58, 132), (132, 93), (93, 234), (234, 127), (127, 162), 
            (162, 21), (21, 54), (54, 103), (103, 67), (67, 109), (109, 10)
        ]
        
        left_eye_connections = [
            (33, 7), (7, 163), (163, 144), (144, 145), (145, 153), 
            (153, 154), (154, 155), (155, 133), (133, 173), (173, 157), 
            (157, 158), (158, 159), (159, 160), (160, 161), (161, 246), 
            (246, 33)
        ]
        
        right_eye_connections = [
            (263, 249), (249, 390), (390, 373), (373, 374), (374, 380), 
            (380, 381), (381, 382), (382, 362), (362, 398), (398, 384), 
            (384, 385), (385, 386), (386, 387), (387, 388), (388, 466), 
            (466, 263)
        ]
        
        lips_outer_connections = [
            (61, 146), (146, 91), (91, 181), (181, 84), (84, 17), 
            (17, 314), (314, 405), (405, 321), (321, 375), (375, 291), 
            (291, 61)
        ]
        
        lips_inner_connections = [
            (78, 95), (95, 88), (88, 178), (178, 87), (87, 14), 
            (14, 317), (317, 402), (402, 318), (318, 324), (324, 308), 
            (308, 78)
        ]
        
        nose_line = [
            (1, 168)
        ]
        
        for connection_set in [outline_connections, left_eye_connections, 
                              right_eye_connections, lips_outer_connections, 
                              lips_inner_connections, nose_line]:
            for connection in connection_set:
                start_idx = connection[0]
                end_idx = connection[1]
                
                start_point = results.face_landmarks.landmark[start_idx]
                end_point = results.face_landmarks.landmark[end_idx]
                
                start_x = int(start_point.x * image_width)
                start_y = int(start_point.y * image_height)
                end_x = int(end_point.x * image_width)
                end_y = int(end_point.y * image_height)
                
                line_color = COLORS["face_color"]
                line_thickness = 3
                
                if connection_set is nose_line:
                    line_color = COLORS["face_color"]
                    line_thickness = 5
                
                cv2.line(image, (start_x, start_y), (end_x, end_y), line_color, line_thickness)
    
    return image

def visualize_colored_landmarks(class_names, video_label=None):
    if not video_label:
        video_label = globals().get('video_label', None)
        if not video_label:
            return
    
    # Clear ALL displays before processing new input
    clear_image_display()
    video_label.config(image=None)
    
    if not class_names:
        status_label.config(text="No valid signs found", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        return
    
    status_label.config(text="Processing...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
    root.update()
    
    # Process each sign one by one
    for class_name in class_names:
        # Clear image display before each new word
        clear_image_display()
        
        # Try to display image for this word and check if successful
        image_found = display_image_for_word(class_name)
        
        # If no image was found, update the label to indicate that
        if not image_found:
            root.word_image_label.config(text=f"No image available for '{class_name}'")
        
        # Process sign language video
        folder_path = os.path.join(DATASET_PATH, class_name)
        if not os.path.exists(folder_path):
            continue
        
        video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        if not video_files:
            continue

        video_path = os.path.join(folder_path, video_files[0])
        cap = cv2.VideoCapture(video_path)

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(frame)
                
                black_frame = np.zeros(frame.shape, dtype=np.uint8)
                black_frame = draw_colored_landmarks(black_frame, results)
                
                frame_resized = cv2.resize(black_frame, (video_label.winfo_width(), video_label.winfo_height()))
                img = Image.fromarray(frame_resized)
                imgtk = ImageTk.PhotoImage(image=img)

                video_label.imgtk = imgtk
                video_label.configure(image=imgtk)
                video_label.update()

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
    
    if class_names:
        status_label.config(text="Translation Complete", font=("Poppins", 14, "bold"), foreground=COLORS["secondary"])
    else:
        status_label.config(text="No valid signs found", font=("Poppins", 14, "bold"), foreground=COLORS["error"])

def on_visualize(event=None):
    input_text = class_entry.get("1.0", tk.END).strip()
    
    # Clear ALL displays before processing new input
    clear_image_display()
    if hasattr(root, 'video_label'):
        root.video_label.config(image=None)
    
    valid_classes = get_valid_classes(input_text)

    if valid_classes:
        status_label.config(text="Processing...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        root.update()
        visualize_colored_landmarks(valid_classes)
    else:
        status_label.config(text="No valid sign found", font=("Poppins", 14, "bold"), foreground=COLORS["error"])
        clear_image_display()


In [50]:

# Calculator Functions
def calculate():
    """Perform the calculation and display the result."""
    try:
        expression = calc_display.get()
        result = str(eval(expression))
        
        # Update calculator display
        calc_display.delete(0, tk.END)
        calc_display.insert(0, result)
        
        # Update the calculation visualization
        update_calculation_visualization(expression, result)
        
    except Exception as e:
        calc_display.delete(0, tk.END)
        calc_display.insert(0, "Error")
        status_label.config(text="Invalid calculation", font=("Poppins", 14, "bold"), foreground=COLORS["error"])

def translate_calculator_input(input_str):
    """Translate calculator input into sign language components."""
    components = []
    current_num = ""
    
    for char in input_str:
        if char.isdigit() or char == '.':
            current_num += char
        else:
            if current_num:
                # Add each digit of the number
                for digit in current_num:
                    if digit == '.':
                        components.append('point')
                    else:
                        components.append(digit)
                current_num = ""
            if char in CALCULATOR_SYMBOLS:
                components.append(CALCULATOR_SYMBOLS[char])
    
    if current_num:
        for digit in current_num:
            if digit == '.':
                components.append('point')
            else:
                components.append(digit)
    
    return components


In [51]:

def update_calculation_visualization(expression, result):
    """Update the calculation visualization boxes and translate the result."""
    # Clear previous visualization
    for widget in calc_visualization_frame.winfo_children():
        widget.destroy()
    
    # Parse the expression into components
    components = []
    current_num = ""
    
    for char in expression:
        if char.isdigit() or char == '.':
            current_num += char
        else:
            if current_num:
                components.append(current_num)
                current_num = ""
            if char in '+-*/=':
                components.append(char)
    
    if current_num:
        components.append(current_num)
    
    # Create visualization boxes with improved styling
    for i, component in enumerate(components):
        if component in '+-*/=':
            # Operator box with accent color
            box = tk.Label(calc_visualization_frame, text=component, 
                          font=("Poppins", 18, "bold"), 
                          bg=COLORS["primary"], fg=COLORS["dark_bg"],
                          relief="flat", borderwidth=0, padx=15, pady=10)
        else:
            # Number box with dark background
            box = tk.Label(calc_visualization_frame, text=component, 
                          font=("Poppins", 18, "bold"), 
                          bg=COLORS["lighter_bg"], fg=COLORS["text_primary"],
                          relief="flat", borderwidth=0, padx=15, pady=10)
        
        box.grid(row=0, column=i, padx=5, pady=5, sticky="nsew")
    
    # Add arrow with improved styling
    arrow = tk.Label(calc_visualization_frame, text="→", 
                     font=("Poppins", 20, "bold"), 
                     bg=COLORS["light_bg"], fg=COLORS["primary"])
    arrow.grid(row=0, column=len(components), padx=10, pady=5)
    
    # Add result box with improved styling
    result_box = tk.Label(calc_visualization_frame, text=result, 
                          font=("Poppins", 18, "bold"), 
                          bg=COLORS["primary"], fg=COLORS["dark_bg"],
                          relief="flat", borderwidth=0, padx=15, pady=10)
    result_box.grid(row=0, column=len(components)+1, padx=5, pady=5, sticky="nsew")
    
    # Translate the result into sign language
    result_str = str(result)
    if '.' in result_str:
        # Handle decimal numbers
        integer_part, decimal_part = result_str.split('.')
        result_classes = []
        for digit in integer_part:
            result_classes.append(digit.lower())
        result_classes.append('point')
        for digit in decimal_part:
            result_classes.append(digit.lower())
    else:
        result_classes = [digit.lower() for digit in result_str]
    
    # Visualize the result in the calculator tab's video label
    if result_classes:
        status_label.config(text="Translating result...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        root.update()
        visualize_colored_landmarks(result_classes, calc_video_label)

def on_calc_button_click(char):
    """Handle calculator button clicks."""
    current = calc_display.get()
    
    if char == "C":
        calc_display.delete(0, tk.END)
        # Clear visualization
        for widget in calc_visualization_frame.winfo_children():
            widget.destroy()
        # Clear video output
        calc_video_label.config(image=None, text="Sign Language Output")
    elif char == "=":
        calculate()
    else:
        calc_display.insert(tk.END, char)
        # Only translate the most recent input
        if char.isdigit() or char == '.':
            # For numbers, translate the digit
            translate_current_calculator_input([char.lower()])
        else:
            # For operators, translate the operator
            translate_current_calculator_input([CALCULATOR_SYMBOLS.get(char, char)])
            
            


In [52]:

def translate_current_calculator_input(components):
    """Translate the current calculator input to sign language."""
    if not components:
        return
    
    if components:
        status_label.config(text="Translating input...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        root.update()
        visualize_colored_landmarks(components, calc_video_label)

def create_calculator(parent):
    """Create the calculator interface with split view."""
    # Create a container frame for the split view
    container = ttk.Frame(parent)
    container.pack(fill=tk.BOTH, expand=True)
    
    # Left side - Calculator interface
    calc_frame = ttk.Frame(container, padding=20, style="TFrame")
    calc_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    # Calculator display with improved styling
    global calc_display
    calc_display = tk.Entry(calc_frame, font=("Poppins", 28, "bold"), 
                          justify="right", bg=COLORS["calc_display"], fg=COLORS["text_primary"],
                          insertbackground=COLORS["text_primary"], borderwidth=0, relief="flat",
                          highlightthickness=2, highlightbackground=COLORS["primary"],
                          highlightcolor=COLORS["primary"])
    calc_display.grid(row=0, column=0, columnspan=4, sticky="nsew", pady=(0, 20), ipady=15)
    
    # Calculator buttons with improved styling
    buttons = [
        ('7', '8', '9', '/'),
        ('4', '5', '6', '*'),
        ('1', '2', '3', '-'),
        ('C', '0', '=', '+')
    ]
    
    for i, row in enumerate(buttons):
        for j, char in enumerate(row):
            # Determine button style based on character
            if char == '=':
                bg = COLORS["calc_button_eq"]
                fg = COLORS["dark_bg"]
                active_bg = "#9A67EA"  # Lighter purple
            elif char == 'C':
                bg = COLORS["calc_button_clear"]
                fg = COLORS["text_primary"]
                active_bg = "#B74D54"  # Lighter red
            elif char in '+-*/':
                bg = COLORS["calc_button_op"]
                fg = COLORS["dark_bg"]
                active_bg = "#00BFA5"  # Lighter teal
            else:
                bg = COLORS["calc_button_num"]
                fg = COLORS["text_primary"]
                active_bg = "#424242"  # Lighter gray
            
            btn = tk.Button(calc_frame, text=char, 
                          command=lambda c=char: on_calc_button_click(c),
                          font=("Poppins", 18, "bold"),
                          bg=bg, fg=fg,
                          activebackground=active_bg,
                          activeforeground=fg,
                          relief="flat", borderwidth=0,
                          width=5, height=2)
            btn.grid(row=i+1, column=j, sticky="nsew", padx=5, pady=5)
    
    # Configure grid weights
    for i in range(4):
        calc_frame.grid_columnconfigure(i, weight=1)
    for i in range(5):
        calc_frame.grid_rowconfigure(i, weight=1)
    
    # Calculation visualization frame with improved styling
    global calc_visualization_frame
    calc_visualization_frame = tk.Frame(calc_frame, bg=COLORS["light_bg"], padx=10, pady=10)
    calc_visualization_frame.grid(row=6, column=0, columnspan=4, sticky="nsew", pady=(20, 0))
    
    # Label for visualization with improved styling
    viz_label = tk.Label(calc_visualization_frame, text="Calculation Steps:", 
                        font=("Poppins", 14, "bold"), bg=COLORS["light_bg"], fg=COLORS["primary"])
    viz_label.pack(anchor="w", pady=(0, 10))
    
    # Right side - Video output for calculator
    video_frame = ttk.Frame(container, padding=10, style="TFrame")
    video_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    # Video label for calculator tab with improved styling
    global calc_video_label
    calc_video_label = tk.Label(video_frame, text="Sign Language Output", 
                               font=("Poppins", 14, "bold"), 
                               bg=COLORS["light_bg"], fg=COLORS["text_secondary"])
    calc_video_label.pack(fill=tk.BOTH, expand=True)
    
    


In [53]:

def listen_to_speech():
    recognizer = sr.Recognizer()
    
    try:
        with sr.Microphone() as source:
            mic_button.config(text="🎤 Listening...", style="Listening.TButton")
            status_label.config(text="Listening... Speak now", font=("Poppins", 18, "bold"), foreground=COLORS["secondary"])
            root.update()
            
            recognizer.adjust_for_ambient_noise(source, duration=1)
            
            try:
                audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
                try:
                    text = recognizer.recognize_google(audio)
                    speech_queue.put(("success", text))
                    
                    class_entry.delete("1.0", tk.END)
                    class_entry.insert("1.0", text)
                    
                    # Clear displays before processing new input
                    clear_image_display()
                    if hasattr(root, 'video_label'):
                        root.video_label.config(image=None)
                    
                    valid_classes = get_valid_classes(text)
                    if valid_classes:
                        status_label.config(text="Processing speech...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
                        root.update()
                        visualize_colored_landmarks(valid_classes)
                    else:
                        status_label.config(text="No valid sign found in speech", font=("Poppins", 14, "bold"), foreground=COLORS["error"])
                        clear_image_display()
                        
                except sr.UnknownValueError:
                    speech_queue.put(("error", "Could not understand audio"))
                except sr.RequestError as e:
                    speech_queue.put(("error", f"Speech service error: {e}"))
            except sr.WaitTimeoutError:
                speech_queue.put(("error", "Listening timed out"))
    except OSError as e:
        speech_queue.put(("error", f"Microphone error: {e}"))
    finally:
        speech_queue.put(("reset_button", ""))

def start_speech_recognition():
    mic_button.config(state=tk.DISABLED)
    threading.Thread(target=listen_to_speech, daemon=True).start()
    root.after(100, check_speech_queue)

def check_speech_queue():
    try:
        while not speech_queue.empty():
            result_type, content = speech_queue.get_nowait()
            
            if result_type == "success":
                status_label.config(text="Speech translated successfully", font=("Poppins", 14, "bold"), foreground=COLORS["secondary"])
            elif result_type == "error":
                status_label.config(text=content, font=("Poppins", 14, "bold"), foreground=COLORS["error"])
            elif result_type == "reset_button":
                mic_button.config(text="🎤 Speak", style="TButton", state=tk.NORMAL)
    except:
        pass
    
    root.after(100, check_speech_queue)
    
    


In [54]:

# GUI Setup
root = tk.Tk()
root.title("Sign Language Translator")
root.geometry("1200x900")
root.configure(bg=COLORS["dark_bg"])

# Custom Fonts
title_font = tkFont.Font(family="Poppins", size=24, weight="bold")
label_font = tkFont.Font(family="Poppins", size=14, weight="bold")
button_font = tkFont.Font(family="Poppins", size=14, weight="bold")
status_font = tkFont.Font(family="Poppins", size=12)
entry_font = tkFont.Font(family="Poppins", size=12)

# Custom Styles
style = ttk.Style(root)
style.theme_use("clam")

style.configure("TFrame", background=COLORS["light_bg"])
style.configure("Title.TFrame", background=COLORS["dark_bg"])
style.configure("TLabel", background=COLORS["light_bg"], foreground=COLORS["text_primary"], font=label_font)
style.configure("Title.TLabel", background=COLORS["dark_bg"], foreground=COLORS["primary"], font=title_font)
style.configure("TButton", background=COLORS["primary"], foreground=COLORS["dark_bg"], 
                font=button_font, borderwidth=0, focusthickness=3, focuscolor=COLORS["primary"])
style.map("TButton", 
          background=[("active", COLORS["primary_variant"])],
          foreground=[("active", COLORS["dark_bg"])])
style.configure("Listening.TButton", background=COLORS["error"], foreground=COLORS["text_primary"])

# Configure Notebook Style
style.configure("TNotebook", background=COLORS["dark_bg"], borderwidth=0)
style.configure("TNotebook.Tab", 
                background=COLORS["darker_bg"], 
                foreground=COLORS["text_secondary"],
                padding=[15, 5],
                font=("Poppins", 12, "bold"))
style.map("TNotebook.Tab", 
          background=[("selected", COLORS["light_bg"])],
          foreground=[("selected", COLORS["primary"])])

# Title Bar
title_bar = ttk.Frame(root, style="Title.TFrame")
title_bar.pack(fill=tk.X, side=tk.TOP, pady=(5, 0))

title_label = ttk.Label(title_bar, text="Sign Language Translator & Calculator", style="")
title_label.pack(pady=(5, 5))

# Main Frame
main_frame = ttk.Frame(root, padding="0")
main_frame.pack(fill=tk.BOTH, expand=True)

# Create notebook for tabs
notebook = ttk.Notebook(main_frame)
notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

# Tab 1: Text to Sign Translation
translation_tab = ttk.Frame(notebook, padding="10")
notebook.add(translation_tab, text="Text to Sign")

layout_frame = ttk.Frame(translation_tab)
layout_frame.pack(fill=tk.BOTH, expand=True)

# Left Section (Input + Image)
left_frame = ttk.Frame(layout_frame, padding="20", style="TFrame")
left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=8, pady=8)

# Input Section
input_frame = ttk.Frame(left_frame, style="TFrame")
input_frame.pack(fill=tk.BOTH, expand=True)

class_label = ttk.Label(input_frame, text="Enter Text:", style="TLabel")
class_label.pack(pady=(0, 10))

class_entry = tk.Text(input_frame, font=("Poppins", 16, "bold"), height=14, width=30, wrap="word", 
                     bg=COLORS["lighter_bg"], fg=COLORS["text_primary"], 
                     insertbackground=COLORS["text_primary"],
                     highlightthickness=1, highlightbackground=COLORS["primary"],
                     highlightcolor=COLORS["primary"], relief="flat")
class_entry.pack(pady=10, ipady=10, fill=tk.BOTH, expand=True)

class_entry.bind("<Return>", on_visualize)

button_frame = ttk.Frame(input_frame)
button_frame.pack(pady=10)

visualize_button = ttk.Button(button_frame, text="Translate", command=on_visualize, style="TButton")
visualize_button.pack(side=tk.LEFT, padx=5)

mic_button = ttk.Button(button_frame, text="🎤 Speak", command=start_speech_recognition, style="TButton")
mic_button.pack(side=tk.LEFT, padx=5)

status_label = ttk.Label(input_frame, text="", style="TLabel", anchor="w")
status_label.pack(pady=10)

# Image display frame (below input section)
image_frame = ttk.Frame(left_frame, height=500, style="TFrame")
image_frame.pack(fill=tk.BOTH, expand=True)

# Initialize image display labels
root.word_image_label = ttk.Label(image_frame, text="", style="TLabel")
root.word_image_label.pack(pady=(0, 10))
root.word_image_display = ttk.Label(image_frame)
root.word_image_display.pack(pady=(0, 10))

# Right Section (Video Output)
output_frame = ttk.Frame(layout_frame, padding="10", style="TFrame")
output_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

video_label = ttk.Label(output_frame, text="Sign Language Output", style="TLabel")
video_label.pack(fill=tk.BOTH, expand=True)

# Tab 2: Calculator
calculator_tab = ttk.Frame(notebook, padding="10")
notebook.add(calculator_tab, text="Calculator")
create_calculator(calculator_tab)

# Run the application
root.mainloop()

In [ ]:
import os
import cv2
import numpy as np
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import string
import mediapipe as mp
import difflib
import spacy
from tkinter import font as tkFont
import speech_recognition as sr
import threading
from queue import Queue
import time
import re
from collections import defaultdict

# MediaPipe and Dataset Setup
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

DATASET_PATH = r"D:\ROHIT\1400+data"
IMAGE_DATASET_PATH = r"D:\ROHIT\Images_path"
nlp = spacy.load("en_core_web_sm")

# Stop Words and Speech Recognition Queue
STOP_WORDS = {
    'a', 'an', 'the', 'was', 'if', 'for', 'and', 'or', 'but', 
    'with', 'are', 'is', 'in', 'on', 'at', 'to', 'of', 'by', 
    'as', 'it', 'this', 'that', 'these', 'those', '?', '!'
}

NEGATIONS = {"not", "no", "never", "none", "n't"}
QUESTION_WORDS = {'who', 'what', 'where', 'when', 'why', 'how', 'which', 'whom', 'whose'}

speech_queue = Queue()

# Calculator operations mapping
CALCULATOR_SYMBOLS = {
    '+': 'addition',
    '-': 'subtract',
    '*': 'multiply',
    '/': 'divide',
    '=': 'equal',
    '.': 'dot'
}

# Color Scheme
COLORS = {
    "dark_bg": "#121212",
    "darker_bg": "#0A0A0A",
    "light_bg": "#1E1E1E",
    "lighter_bg": "#2E2E2E",
    "primary": "#BB86FC",
    "primary_variant": "#3700B3",
    "secondary": "#03DAC6",
    "error": "#CF6679",
    "text_primary": "#FFFFFF",
    "text_secondary": "#B3B3B3",
    "button_bg": "#333333",
    "button_active": "#424242",
    "pose_color": (0, 255, 0),
    "left_hand_color": (255, 0, 0),
    "right_hand_color": (255, 0, 255),
    "face_color": (255, 69, 126),
    "connection_color": (255, 255, 0),
    "neck_color": (0, 255, 255),
    "background_color": (0, 0, 0),
    "calc_display": "#1E1E1E",
    "calc_button_num": "#333333",
    "calc_button_op": "#03DAC6",
    "calc_button_eq": "#BB86FC",
    "calc_button_clear": "#CF6679"
}

def build_word_variations_map():
    variations_map = {
        'you': {'your', 'yours', "you're", "you've", "you'll"},
        'my': {'me', 'my', 'i am ', "i'm", 'myself', 'mine'},
        'mother': {'She', 'her'},
        'father': {'He', 'him'},
        'name': {'names', 'named'},
        'what': {"what's", "what're"},
        'eat': {'ate', 'eating', 'eaten'},
        'run': {'ran', 'running'},
    }
    
    variations_map_copy = defaultdict(set)
    for root, variations in variations_map.items():
        variations_map_copy[root] = variations.copy()
        if root.endswith(('s', 'x', 'z', 'ch', 'sh')):
            plural = root + 'es'
        else:
            plural = root + 's'
        variations_map_copy[root].add(plural)
    
    return dict(variations_map_copy)

WORD_VARIATIONS_MAP = build_word_variations_map()

def split_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

def convert_to_isl(english_sentence):
    doc = nlp(english_sentence.lower())
    
    subject = []
    obj = []
    verb = []
    adjectives = []
    question_word = []
    negation = []
    
    for token in doc:
        word = token.text
        if word in STOP_WORDS or token.is_punct:
            continue
            
        if word in QUESTION_WORDS:
            question_word.append(word.upper())
            continue
            
        if word in NEGATIONS:
            negation.append(word.upper())
            continue
            
        if "subj" in token.dep_:
            subject.append(word.upper())
            continue
            
        if "obj" in token.dep_:
            obj.append(word.upper())
            continue
            
        if token.pos_ == "VERB":
            verb.append(token.lemma_.upper())
            continue
            
        if token.pos_ == "ADJ":
            adjectives.append(word.upper())
            continue
            
        obj.append(word.upper())
    
    isl_gloss = subject + obj + verb + adjectives + question_word + negation
    return " ".join(isl_gloss)

def process_multiple_sentences(text):
    sentences = split_sentences(text)
    return [convert_to_isl(sent) for sent in sentences]

def get_valid_classes(input_text):
    if not os.path.exists(DATASET_PATH):
        return []
    
    isl_sentences = process_multiple_sentences(input_text)
    available_classes = os.listdir(DATASET_PATH)
    available_classes_lower = {label.lower().strip(): label for label in available_classes}
    
    matched_classes = []
    
    for isl_text in isl_sentences:
        input_text = isl_text.strip().lower()
        
        if not input_text or input_text in STOP_WORDS:
            continue
        
        if input_text in available_classes_lower:
            matched_classes.append(available_classes_lower[input_text])
            continue
        
        words = input_text.split()
        for phrase_length in range(4, 1, -1):
            for i in range(len(words) - phrase_length + 1):
                phrase = ' '.join(words[i:i+phrase_length])
                if phrase in available_classes_lower:
                    before_phrase = ' '.join(words[:i])
                    after_phrase = ' '.join(words[i+phrase_length:])
                    
                    result = []
                    if before_phrase:
                        result.extend(get_valid_classes(before_phrase))
                    result.append(available_classes_lower[phrase])
                    if after_phrase:
                        result.extend(get_valid_classes(after_phrase))
                    
                    matched_classes.extend(result)
                    break
            else:
                continue
            break
        else:
            components = []
            current_word = ""
            for char in input_text:
                if char.isalpha() or char.isdigit():
                    current_word += char
                else:
                    if current_word:
                        components.append(current_word)
                        current_word = ""
                    if char.strip():
                        components.append(char)
            if current_word:
                components.append(current_word)
            
            for component in components:
                if not component.strip() or component.lower() in STOP_WORDS:
                    continue
                    
                if component.isdigit():
                    for digit in component:
                        if digit.lower() in available_classes_lower:
                            matched_classes.append(available_classes_lower[digit.lower()])
                    continue
                    
                if len(component) == 1 and component.isalpha():
                    if component.lower() in available_classes_lower:
                        matched_classes.append(available_classes_lower[component.lower()])
                    continue
                    
                if len(component) > 1 and component.isalpha():
                    lower_component = component.lower()
                    
                    if lower_component in available_classes_lower:
                        matched_classes.append(available_classes_lower[lower_component])
                        continue
                        
                    doc = nlp(lower_component)
                    lemma = doc[0].lemma_ if doc else lower_component
                    
                    if lemma in available_classes_lower:
                        matched_classes.append(available_classes_lower[lemma])
                        continue
                        
                    found_variation = False
                    for root_word, variations in WORD_VARIATIONS_MAP.items():
                        if lower_component in variations and root_word in available_classes_lower:
                            matched_classes.append(available_classes_lower[root_word])
                            found_variation = True
                            break
                            
                    if found_variation:
                        continue
                        
                    for letter in component.upper():
                        letter_lower = letter.lower()
                        if letter_lower in available_classes_lower:
                            matched_classes.append(available_classes_lower[letter_lower])
                    continue
                    
                lower_component = component.lower()
                if lower_component in available_classes_lower:
                    matched_classes.append(available_classes_lower[lower_component])
                    continue
                    
                doc = nlp(component.lower())
                lemmatized_words = [token.lemma_ for token in doc if token.text not in STOP_WORDS]
                
                for word in lemmatized_words:
                    if word in available_classes_lower:
                        matched_classes.append(available_classes_lower[word])
                        continue
                        
                    for root_word, variations in WORD_VARIATIONS_MAP.items():
                        if word in variations and root_word in available_classes_lower:
                            matched_classes.append(available_classes_lower[root_word])
                            continue
                            
                    closest_match = difflib.get_close_matches(word, available_classes_lower.keys(), n=1, cutoff=0.6)
                    if closest_match:
                        matched_classes.append(available_classes_lower[closest_match[0]])
    
    return matched_classes

def clear_image_display():
    """Clear the image display completely"""
    if hasattr(root, 'word_image_display'):
        root.word_image_display.config(image=None)
        root.word_image_label.config(text="")

def display_image_for_word(word):
    """Display image only if it exists in the dataset"""
    # First clear any existing image
    clear_image_display()
    
    if not os.path.exists(IMAGE_DATASET_PATH):
        return False
    
    clean_word = word.strip().lower()
    clean_word = ''.join([c for c in clean_word if c.isalpha() or c.isdigit()])
    
    if not clean_word:
        return False
    
    # Only show images for single words
    if len(clean_word.split()) > 1:
        return False
    
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']
    image_path = None
    
    for ext in image_extensions:
        potential_path = os.path.join(IMAGE_DATASET_PATH, f"{clean_word}{ext}")
        if os.path.exists(potential_path):
            image_path = potential_path
            break
    
    if not image_path:
        return False
    
    try:
        img = Image.open(image_path)
        img = img.resize((300, 300), Image.LANCZOS)
        
        if not hasattr(root, 'word_image_label'):
            root.word_image_label = ttk.Label(image_frame, text="Image:", style="TLabel")
            root.word_image_label.pack(pady=(0, 5))
            
            root.word_image_display = ttk.Label(image_frame)
            root.word_image_display.pack(pady=(0, 10))
        
        imgtk = ImageTk.PhotoImage(image=img)
        root.word_image_display.imgtk = imgtk
        root.word_image_display.config(image=imgtk)
        root.word_image_label.config(text=f"Image for '{clean_word}':")
        return True
    except Exception as e:
        print(f"Error loading image: {e}")
        clear_image_display()
        return False

def draw_colored_landmarks(image, results):
    image_height, image_width, _ = image.shape
    
    if results.pose_landmarks:
        POSE_CONNECTIONS_TO_KEEP = [
            (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_SHOULDER),
            (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.LEFT_ELBOW),
            (mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.LEFT_WRIST),
            (mp_holistic.PoseLandmark.RIGHT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_ELBOW),
            (mp_holistic.PoseLandmark.RIGHT_ELBOW, mp_holistic.PoseLandmark.RIGHT_WRIST),
            (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.LEFT_HIP),
            (mp_holistic.PoseLandmark.RIGHT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_HIP),
            (mp_holistic.PoseLandmark.LEFT_HIP, mp_holistic.PoseLandmark.RIGHT_HIP),
            (mp_holistic.PoseLandmark.LEFT_HIP, mp_holistic.PoseLandmark.LEFT_KNEE),
            (mp_holistic.PoseLandmark.LEFT_KNEE, mp_holistic.PoseLandmark.LEFT_ANKLE),
            (mp_holistic.PoseLandmark.RIGHT_HIP, mp_holistic.PoseLandmark.RIGHT_KNEE),
            (mp_holistic.PoseLandmark.RIGHT_KNEE, mp_holistic.PoseLandmark.RIGHT_ANKLE)
        ]
        
        for connection in POSE_CONNECTIONS_TO_KEEP:
            start_idx = connection[0].value
            end_idx = connection[1].value
            
            start_point = results.pose_landmarks.landmark[start_idx]
            end_point = results.pose_landmarks.landmark[end_idx]
            
            start_x = int(start_point.x * image_width)
            start_y = int(start_point.y * image_height)
            end_x = int(end_point.x * image_width)
            end_y = int(end_point.y * image_height)
            
            cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["pose_color"], 10)
        
        if results.face_landmarks:
            left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value]
            chin = results.face_landmarks.landmark[152]
            
            neck_x = int(((left_shoulder.x + right_shoulder.x) / 2) * image_width)
            neck_y = int(((left_shoulder.y + right_shoulder.y) / 2) * image_height)
            
            chin_x = int(chin.x * image_width)
            chin_y = int(chin.y * image_height)
            
            cv2.line(image, (neck_x, neck_y), (chin_x, chin_y), COLORS["neck_color"], 10)
    
    if results.left_hand_landmarks:
        for connection in mp_holistic.HAND_CONNECTIONS:
            start_idx = connection[0]
            end_idx = connection[1]
            
            start_point = results.left_hand_landmarks.landmark[start_idx]
            end_point = results.left_hand_landmarks.landmark[end_idx]
            
            start_x = int(start_point.x * image_width)
            start_y = int(start_point.y * image_height)
            end_x = int(end_point.x * image_width)
            end_y = int(end_point.y * image_height)
            
            cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["left_hand_color"], 4)
    
    if results.right_hand_landmarks:
        for connection in mp_holistic.HAND_CONNECTIONS:
            start_idx = connection[0]
            end_idx = connection[1]
            
            start_point = results.right_hand_landmarks.landmark[start_idx]
            end_point = results.right_hand_landmarks.landmark[end_idx]
            
            start_x = int(start_point.x * image_width)
            start_y = int(start_point.y * image_height)
            end_x = int(end_point.x * image_width)
            end_y = int(end_point.y * image_height)
            
            cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["right_hand_color"], 4)
    
    if results.face_landmarks:
        outline_connections = [
            (10, 338), (338, 297), (297, 332), (332, 284), (284, 251), 
            (251, 389), (389, 356), (356, 454), (454, 323), (323, 361), 
            (361, 288), (288, 397), (397, 365), (365, 379), (379, 378), 
            (378, 400), (400, 377), (377, 152), (152, 148), (148, 176), 
            (176, 149), (149, 150), (150, 136), (136, 172), (172, 58), 
            (58, 132), (132, 93), (93, 234), (234, 127), (127, 162), 
            (162, 21), (21, 54), (54, 103), (103, 67), (67, 109), (109, 10)
        ]
        
        left_eye_connections = [
            (33, 7), (7, 163), (163, 144), (144, 145), (145, 153), 
            (153, 154), (154, 155), (155, 133), (133, 173), (173, 157), 
            (157, 158), (158, 159), (159, 160), (160, 161), (161, 246), 
            (246, 33)
        ]
        
        right_eye_connections = [
            (263, 249), (249, 390), (390, 373), (373, 374), (374, 380), 
            (380, 381), (381, 382), (382, 362), (362, 398), (398, 384), 
            (384, 385), (385, 386), (386, 387), (387, 388), (388, 466), 
            (466, 263)
        ]
        
        lips_outer_connections = [
            (61, 146), (146, 91), (91, 181), (181, 84), (84, 17), 
            (17, 314), (314, 405), (405, 321), (321, 375), (375, 291), 
            (291, 61)
        ]
        
        lips_inner_connections = [
            (78, 95), (95, 88), (88, 178), (178, 87), (87, 14), 
            (14, 317), (317, 402), (402, 318), (318, 324), (324, 308), 
            (308, 78)
        ]
        
        nose_line = [
            (1, 168)
        ]
        
        for connection_set in [outline_connections, left_eye_connections, 
                              right_eye_connections, lips_outer_connections, 
                              lips_inner_connections, nose_line]:
            for connection in connection_set:
                start_idx = connection[0]
                end_idx = connection[1]
                
                start_point = results.face_landmarks.landmark[start_idx]
                end_point = results.face_landmarks.landmark[end_idx]
                
                start_x = int(start_point.x * image_width)
                start_y = int(start_point.y * image_height)
                end_x = int(end_point.x * image_width)
                end_y = int(end_point.y * image_height)
                
                line_color = COLORS["face_color"]
                line_thickness = 3
                
                if connection_set is nose_line:
                    line_color = COLORS["face_color"]
                    line_thickness = 5
                
                cv2.line(image, (start_x, start_y), (end_x, end_y), line_color, line_thickness)
    
    return image

def visualize_colored_landmarks(class_names, video_label=None):
    if not video_label:
        video_label = globals().get('video_label', None)
        if not video_label:
            return
    
    # Clear ALL displays before processing new input
    clear_image_display()
    video_label.config(image=None)
    
    if not class_names:
        status_label.config(text="No valid signs found", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        return
    
    status_label.config(text="Processing...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
    root.update()
    
    # Process each sign one by one
    for class_name in class_names:
        # Clear image display before each new word
        clear_image_display()
        
        # Try to display image for this word and check if successful
        image_found = display_image_for_word(class_name)
        
        # If no image was found, update the label to indicate that
        if not image_found:
            root.word_image_label.config(text=f"No image available for '{class_name}'")
        
        # Process sign language video
        folder_path = os.path.join(DATASET_PATH, class_name)
        if not os.path.exists(folder_path):
            continue
        
        video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
        if not video_files:
            continue

        video_path = os.path.join(folder_path, video_files[0])
        cap = cv2.VideoCapture(video_path)

        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = holistic.process(frame)
                
                black_frame = np.zeros(frame.shape, dtype=np.uint8)
                black_frame = draw_colored_landmarks(black_frame, results)
                
                frame_resized = cv2.resize(black_frame, (video_label.winfo_width(), video_label.winfo_height()))
                img = Image.fromarray(frame_resized)
                imgtk = ImageTk.PhotoImage(image=img)

                video_label.imgtk = imgtk
                video_label.configure(image=imgtk)
                video_label.update()

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

        cap.release()
    
    if class_names:
        status_label.config(text="Translation Complete", font=("Poppins", 14, "bold"), foreground=COLORS["secondary"])
    else:
        status_label.config(text="No valid signs found", font=("Poppins", 14, "bold"), foreground=COLORS["error"])

def on_visualize(event=None):
    input_text = class_entry.get("1.0", tk.END).strip()
    
    # Clear ALL displays before processing new input
    clear_image_display()
    if hasattr(root, 'video_label'):
        root.video_label.config(image=None)
    
    valid_classes = get_valid_classes(input_text)

    if valid_classes:
        status_label.config(text="Processing...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        root.update()
        visualize_colored_landmarks(valid_classes)
    else:
        status_label.config(text="No valid sign found", font=("Poppins", 14, "bold"), foreground=COLORS["error"])
        clear_image_display()

# Calculator Functions
def calculate():
    """Perform the calculation and display the result."""
    try:
        expression = calc_display.get()
        result = str(eval(expression))
        
        # Update calculator display
        calc_display.delete(0, tk.END)
        calc_display.insert(0, result)
        
        # Update the calculation visualization
        update_calculation_visualization(expression, result)
        
    except Exception as e:
        calc_display.delete(0, tk.END)
        calc_display.insert(0, "Error")
        status_label.config(text="Invalid calculation", font=("Poppins", 14, "bold"), foreground=COLORS["error"])

def translate_calculator_input(input_str):
    """Translate calculator input into sign language components."""
    components = []
    current_num = ""
    
    for char in input_str:
        if char.isdigit() or char == '.':
            current_num += char
        else:
            if current_num:
                # Add each digit of the number
                for digit in current_num:
                    if digit == '.':
                        components.append('point')
                    else:
                        components.append(digit)
                current_num = ""
            if char in CALCULATOR_SYMBOLS:
                components.append(CALCULATOR_SYMBOLS[char])
    
    if current_num:
        for digit in current_num:
            if digit == '.':
                components.append('point')
            else:
                components.append(digit)
    
    return components

def update_calculation_visualization(expression, result):
    """Update the calculation visualization boxes and translate the result."""
    # Clear previous visualization
    for widget in calc_visualization_frame.winfo_children():
        widget.destroy()
    
    # Parse the expression into components
    components = []
    current_num = ""
    
    for char in expression:
        if char.isdigit() or char == '.':
            current_num += char
        else:
            if current_num:
                components.append(current_num)
                current_num = ""
            if char in '+-*/=':
                components.append(char)
    
    if current_num:
        components.append(current_num)
    
    # Create visualization boxes with improved styling
    for i, component in enumerate(components):
        if component in '+-*/=':
            # Operator box with accent color
            box = tk.Label(calc_visualization_frame, text=component, 
                          font=("Poppins", 18, "bold"), 
                          bg=COLORS["primary"], fg=COLORS["dark_bg"],
                          relief="flat", borderwidth=0, padx=15, pady=10)
        else:
            # Number box with dark background
            box = tk.Label(calc_visualization_frame, text=component, 
                          font=("Poppins", 18, "bold"), 
                          bg=COLORS["lighter_bg"], fg=COLORS["text_primary"],
                          relief="flat", borderwidth=0, padx=15, pady=10)
        
        box.grid(row=0, column=i, padx=5, pady=5, sticky="nsew")
    
    # Add arrow with improved styling
    arrow = tk.Label(calc_visualization_frame, text="→", 
                     font=("Poppins", 20, "bold"), 
                     bg=COLORS["light_bg"], fg=COLORS["primary"])
    arrow.grid(row=0, column=len(components), padx=10, pady=5)
    
    # Add result box with improved styling
    result_box = tk.Label(calc_visualization_frame, text=result, 
                          font=("Poppins", 18, "bold"), 
                          bg=COLORS["primary"], fg=COLORS["dark_bg"],
                          relief="flat", borderwidth=0, padx=15, pady=10)
    result_box.grid(row=0, column=len(components)+1, padx=5, pady=5, sticky="nsew")
    
    # Translate the result into sign language
    result_str = str(result)
    if '.' in result_str:
        # Handle decimal numbers
        integer_part, decimal_part = result_str.split('.')
        result_classes = []
        for digit in integer_part:
            result_classes.append(digit.lower())
        result_classes.append('point')
        for digit in decimal_part:
            result_classes.append(digit.lower())
    else:
        result_classes = [digit.lower() for digit in result_str]
    
    # Visualize the result in the calculator tab's video label
    if result_classes:
        status_label.config(text="Translating result...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        root.update()
        visualize_colored_landmarks(result_classes, calc_video_label)

def on_calc_button_click(char):
    """Handle calculator button clicks."""
    current = calc_display.get()
    
    if char == "C":
        calc_display.delete(0, tk.END)
        # Clear visualization
        for widget in calc_visualization_frame.winfo_children():
            widget.destroy()
        # Clear video output
        calc_video_label.config(image=None, text="Sign Language Output")
    elif char == "=":
        calculate()
    else:
        calc_display.insert(tk.END, char)
        # Only translate the most recent input
        if char.isdigit() or char == '.':
            # For numbers, translate the digit
            translate_current_calculator_input([char.lower()])
        else:
            # For operators, translate the operator
            translate_current_calculator_input([CALCULATOR_SYMBOLS.get(char, char)])

def translate_current_calculator_input(components):
    """Translate the current calculator input to sign language."""
    if not components:
        return
    
    if components:
        status_label.config(text="Translating input...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
        root.update()
        visualize_colored_landmarks(components, calc_video_label)

def create_calculator(parent):
    """Create the calculator interface with split view."""
    # Create a container frame for the split view
    container = ttk.Frame(parent)
    container.pack(fill=tk.BOTH, expand=True)
    
    # Left side - Calculator interface
    calc_frame = ttk.Frame(container, padding=20, style="TFrame")
    calc_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    # Calculator display with improved styling
    global calc_display
    calc_display = tk.Entry(calc_frame, font=("Poppins", 28, "bold"), 
                          justify="right", bg=COLORS["calc_display"], fg=COLORS["text_primary"],
                          insertbackground=COLORS["text_primary"], borderwidth=0, relief="flat",
                          highlightthickness=2, highlightbackground=COLORS["primary"],
                          highlightcolor=COLORS["primary"])
    calc_display.grid(row=0, column=0, columnspan=4, sticky="nsew", pady=(0, 20), ipady=15)
    
    # Calculator buttons with improved styling
    buttons = [
        ('7', '8', '9', '/'),
        ('4', '5', '6', '*'),
        ('1', '2', '3', '-'),
        ('C', '0', '=', '+')
    ]
    
    for i, row in enumerate(buttons):
        for j, char in enumerate(row):
            # Determine button style based on character
            if char == '=':
                bg = COLORS["calc_button_eq"]
                fg = COLORS["dark_bg"]
                active_bg = "#9A67EA"  # Lighter purple
            elif char == 'C':
                bg = COLORS["calc_button_clear"]
                fg = COLORS["text_primary"]
                active_bg = "#B74D54"  # Lighter red
            elif char in '+-*/':
                bg = COLORS["calc_button_op"]
                fg = COLORS["dark_bg"]
                active_bg = "#00BFA5"  # Lighter teal
            else:
                bg = COLORS["calc_button_num"]
                fg = COLORS["text_primary"]
                active_bg = "#424242"  # Lighter gray
            
            btn = tk.Button(calc_frame, text=char, 
                          command=lambda c=char: on_calc_button_click(c),
                          font=("Poppins", 18, "bold"),
                          bg=bg, fg=fg,
                          activebackground=active_bg,
                          activeforeground=fg,
                          relief="flat", borderwidth=0,
                          width=5, height=2)
            btn.grid(row=i+1, column=j, sticky="nsew", padx=5, pady=5)
    
    # Configure grid weights
    for i in range(4):
        calc_frame.grid_columnconfigure(i, weight=1)
    for i in range(5):
        calc_frame.grid_rowconfigure(i, weight=1)
    
    # Calculation visualization frame with improved styling
    global calc_visualization_frame
    calc_visualization_frame = tk.Frame(calc_frame, bg=COLORS["light_bg"], padx=10, pady=10)
    calc_visualization_frame.grid(row=6, column=0, columnspan=4, sticky="nsew", pady=(20, 0))
    
    # Label for visualization with improved styling
    viz_label = tk.Label(calc_visualization_frame, text="Calculation Steps:", 
                        font=("Poppins", 14, "bold"), bg=COLORS["light_bg"], fg=COLORS["primary"])
    viz_label.pack(anchor="w", pady=(0, 10))
    
    # Right side - Video output for calculator
    video_frame = ttk.Frame(container, padding=10, style="TFrame")
    video_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    # Video label for calculator tab with improved styling
    global calc_video_label
    calc_video_label = tk.Label(video_frame, text="Sign Language Output", 
                               font=("Poppins", 14, "bold"), 
                               bg=COLORS["light_bg"], fg=COLORS["text_secondary"])
    calc_video_label.pack(fill=tk.BOTH, expand=True)

def listen_to_speech():
    recognizer = sr.Recognizer()
    
    try:
        with sr.Microphone() as source:
            mic_button.config(text="🎤 Listening...", style="Listening.TButton")
            status_label.config(text="Listening... Speak now", font=("Poppins", 18, "bold"), foreground=COLORS["secondary"])
            root.update()
            
            recognizer.adjust_for_ambient_noise(source, duration=1)
            
            try:
                audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
                try:
                    text = recognizer.recognize_google(audio)
                    speech_queue.put(("success", text))
                    
                    class_entry.delete("1.0", tk.END)
                    class_entry.insert("1.0", text)
                    
                    # Clear displays before processing new input
                    clear_image_display()
                    if hasattr(root, 'video_label'):
                        root.video_label.config(image=None)
                    
                    valid_classes = get_valid_classes(text)
                    if valid_classes:
                        status_label.config(text="Processing speech...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
                        root.update()
                        visualize_colored_landmarks(valid_classes)
                    else:
                        status_label.config(text="No valid sign found in speech", font=("Poppins", 14, "bold"), foreground=COLORS["error"])
                        clear_image_display()
                        
                except sr.UnknownValueError:
                    speech_queue.put(("error", "Could not understand audio"))
                except sr.RequestError as e:
                    speech_queue.put(("error", f"Speech service error: {e}"))
            except sr.WaitTimeoutError:
                speech_queue.put(("error", "Listening timed out"))
    except OSError as e:
        speech_queue.put(("error", f"Microphone error: {e}"))
    finally:
        speech_queue.put(("reset_button", ""))

def start_speech_recognition():
    mic_button.config(state=tk.DISABLED)
    threading.Thread(target=listen_to_speech, daemon=True).start()
    root.after(100, check_speech_queue)

def check_speech_queue():
    try:
        while not speech_queue.empty():
            result_type, content = speech_queue.get_nowait()
            
            if result_type == "success":
                status_label.config(text="Speech translated successfully", font=("Poppins", 14, "bold"), foreground=COLORS["secondary"])
            elif result_type == "error":
                status_label.config(text=content, font=("Poppins", 14, "bold"), foreground=COLORS["error"])
            elif result_type == "reset_button":
                mic_button.config(text="🎤 Speak", style="TButton", state=tk.NORMAL)
    except:
        pass
    
    root.after(100, check_speech_queue)

# GUI Setup
root = tk.Tk()
root.title("Sign Language Translator with Calculator")
root.geometry("1400x900")
root.configure(bg=COLORS["dark_bg"])

# Custom Fonts
title_font = tkFont.Font(family="Poppins", size=24, weight="bold")
label_font = tkFont.Font(family="Poppins", size=14, weight="bold")
button_font = tkFont.Font(family="Poppins", size=14, weight="bold")
status_font = tkFont.Font(family="Poppins", size=12)
entry_font = tkFont.Font(family="Poppins", size=12)

# Custom Styles
style = ttk.Style(root)
style.theme_use("clam")

style.configure("TFrame", background=COLORS["light_bg"])
style.configure("Title.TFrame", background=COLORS["dark_bg"])
style.configure("TLabel", background=COLORS["light_bg"], foreground=COLORS["text_primary"], font=label_font)
style.configure("Title.TLabel", background=COLORS["dark_bg"], foreground=COLORS["primary"], font=title_font)
style.configure("TButton", background=COLORS["primary"], foreground=COLORS["dark_bg"], 
                font=button_font, borderwidth=0, focusthickness=3, focuscolor=COLORS["primary"])
style.map("TButton", 
          background=[("active", COLORS["primary_variant"])],
          foreground=[("active", COLORS["dark_bg"])])
style.configure("Listening.TButton", background=COLORS["error"], foreground=COLORS["text_primary"])

# Configure Notebook Style
style.configure("TNotebook", background=COLORS["dark_bg"], borderwidth=0)
style.configure("TNotebook.Tab", 
                background=COLORS["darker_bg"], 
                foreground=COLORS["text_secondary"],
                padding=[15, 5],
                font=("Poppins", 12, "bold"))
style.map("TNotebook.Tab", 
          background=[("selected", COLORS["light_bg"])],
          foreground=[("selected", COLORS["primary"])])

# Title Bar
title_bar = ttk.Frame(root, style="Title.TFrame")
title_bar.pack(fill=tk.X, side=tk.TOP, pady=(10, 0))

title_label = ttk.Label(title_bar, text="Sign Language Translator & Calculator", style="Title.TLabel")
title_label.pack(pady=(10, 20))

# Main Frame
main_frame = ttk.Frame(root, padding="0")
main_frame.pack(fill=tk.BOTH, expand=True)

# Create notebook for tabs
notebook = ttk.Notebook(main_frame)
notebook.pack(fill=tk.BOTH, expand=True, padx=10, pady=(0, 10))

# Tab 1: Text to Sign Translation
translation_tab = ttk.Frame(notebook, padding="10")
notebook.add(translation_tab, text="Text to Sign")

layout_frame = ttk.Frame(translation_tab)
layout_frame.pack(fill=tk.BOTH, expand=True)

# Left Section (Input + Image)
left_frame = ttk.Frame(layout_frame, padding="20", style="TFrame")
left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

# Input Section
input_frame = ttk.Frame(left_frame, style="TFrame")
input_frame.pack(fill=tk.BOTH, expand=True)

class_label = ttk.Label(input_frame, text="Enter Text:", style="TLabel")
class_label.pack(pady=(0, 10))

class_entry = tk.Text(input_frame, font=("Poppins", 16, "bold"), height=14, width=30, wrap="word", 
                     bg=COLORS["lighter_bg"], fg=COLORS["text_primary"], 
                     insertbackground=COLORS["text_primary"],
                     highlightthickness=1, highlightbackground=COLORS["primary"],
                     highlightcolor=COLORS["primary"], relief="flat")
class_entry.pack(pady=10, ipady=10, fill=tk.BOTH, expand=True)

class_entry.bind("<Return>", on_visualize)

button_frame = ttk.Frame(input_frame)
button_frame.pack(pady=10)

visualize_button = ttk.Button(button_frame, text="Translate", command=on_visualize, style="TButton")
visualize_button.pack(side=tk.LEFT, padx=5)

mic_button = ttk.Button(button_frame, text="🎤 Speak", command=start_speech_recognition, style="TButton")
mic_button.pack(side=tk.LEFT, padx=5)

status_label = ttk.Label(input_frame, text="", style="TLabel", anchor="w")
status_label.pack(pady=10)

# Image display frame (below input section)
image_frame = ttk.Frame(left_frame, height=300, style="TFrame")
image_frame.pack(fill=tk.BOTH, expand=False)

# Initialize image display labels
root.word_image_label = ttk.Label(image_frame, text="", style="TLabel")
root.word_image_label.pack(pady=(0, 5))
root.word_image_display = ttk.Label(image_frame)
root.word_image_display.pack(pady=(0, 10))

# Right Section (Video Output)
output_frame = ttk.Frame(layout_frame, padding="10", style="TFrame")
output_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

video_label = ttk.Label(output_frame, text="Sign Language Output", style="TLabel")
video_label.pack(fill=tk.BOTH, expand=True)

# Tab 2: Calculator
calculator_tab = ttk.Frame(notebook, padding="10")
notebook.add(calculator_tab, text="Calculator")
create_calculator(calculator_tab)

# Run the application
root.mainloop()

## without calculator 

In [56]:
# import os
# import cv2
# import numpy as np
# import tkinter as tk
# from tkinter import ttk
# from PIL import Image, ImageTk
# import string
# import mediapipe as mp
# import difflib
# import spacy
# from tkinter import font as tkFont
# import speech_recognition as sr
# import threading
# from queue import Queue
# import time
# import re
# from collections import defaultdict

# # MediaPipe and Dataset Setup
# mp_holistic = mp.solutions.holistic
# mp_drawing = mp.solutions.drawing_utils
# mp_drawing_styles = mp.solutions.drawing_styles

# DATASET_PATH = r"D:\ROHIT\1400+data"
# IMAGE_DATASET_PATH = r"D:\ROHIT\Images_path"
# nlp = spacy.load("en_core_web_sm")

# # Stop Words and Speech Recognition Queue
# STOP_WORDS = {
#     'a', 'an', 'the', 'was', 'if', 'for', 'and', 'or', 'but', 
#     'with', 'are', 'is', 'in', 'on', 'at', 'to', 'of', 'by', 
#     'as', 'it', 'this', 'that', 'these', 'those', '?', '!'
# }

# NEGATIONS = {"not", "no", "never", "none", "n't"}
# QUESTION_WORDS = {'who', 'what', 'where', 'when', 'why', 'how', 'which', 'whom', 'whose'}

# speech_queue = Queue()

# # Color Scheme
# COLORS = {
#     "dark_bg": "#121212",
#     "darker_bg": "#0A0A0A",
#     "light_bg": "#1E1E1E",
#     "lighter_bg": "#2E2E2E",
#     "primary": "#BB86FC",
#     "primary_variant": "#3700B3",
#     "secondary": "#03DAC6",
#     "error": "#CF6679",
#     "text_primary": "#FFFFFF",
#     "text_secondary": "#B3B3B3",
#     "button_bg": "#333333",
#     "button_active": "#424242",
#     "pose_color": (0, 255, 0),
#     "left_hand_color": (255, 0, 0),
#     "right_hand_color": (255, 0, 255),
#     "face_color": (255, 69, 126),
#     "connection_color": (255, 255, 0),
#     "neck_color": (0, 255, 255),
#     "background_color": (0, 0, 0)
# }

# def build_word_variations_map():
#     variations_map = {
#         'you': {'your', 'yours', "you're", "you've", "you'll"},
#         'my': {'me', 'my', 'i am ', "i'm", 'myself', 'mine'},
#         'mother': {'She', 'her'},
#         'father': {'He', 'him'},
#         'name': {'names', 'named'},
#         'what': {"what's", "what're"},
#         'eat': {'ate', 'eating', 'eaten'},
#         'run': {'ran', 'running'},
#     }
    
#     variations_map_copy = defaultdict(set)
#     for root, variations in variations_map.items():
#         variations_map_copy[root] = variations.copy()
#         if root.endswith(('s', 'x', 'z', 'ch', 'sh')):
#             plural = root + 'es'
#         else:
#             plural = root + 's'
#         variations_map_copy[root].add(plural)
    
#     return dict(variations_map_copy)

# WORD_VARIATIONS_MAP = build_word_variations_map()

# def split_sentences(text):
#     doc = nlp(text)
#     return [sent.text.strip() for sent in doc.sents]

# def convert_to_isl(english_sentence):
#     doc = nlp(english_sentence.lower())
    
#     subject = []
#     obj = []
#     verb = []
#     adjectives = []
#     question_word = []
#     negation = []
    
#     for token in doc:
#         word = token.text
#         if word in STOP_WORDS or token.is_punct:
#             continue
            
#         if word in QUESTION_WORDS:
#             question_word.append(word.upper())
#             continue
            
#         if word in NEGATIONS:
#             negation.append(word.upper())
#             continue
            
#         if "subj" in token.dep_:
#             subject.append(word.upper())
#             continue
            
#         if "obj" in token.dep_:
#             obj.append(word.upper())
#             continue
            
#         if token.pos_ == "VERB":
#             verb.append(token.lemma_.upper())
#             continue
            
#         if token.pos_ == "ADJ":
#             adjectives.append(word.upper())
#             continue
            
#         obj.append(word.upper())
    
#     isl_gloss = subject + obj + verb + adjectives + question_word + negation
#     return " ".join(isl_gloss)

# def process_multiple_sentences(text):
#     sentences = split_sentences(text)
#     return [convert_to_isl(sent) for sent in sentences]

# def get_valid_classes(input_text):
#     if not os.path.exists(DATASET_PATH):
#         return []
    
#     isl_sentences = process_multiple_sentences(input_text)
#     available_classes = os.listdir(DATASET_PATH)
#     available_classes_lower = {label.lower().strip(): label for label in available_classes}
    
#     matched_classes = []
    
#     for isl_text in isl_sentences:
#         input_text = isl_text.strip().lower()
        
#         if not input_text or input_text in STOP_WORDS:
#             continue
        
#         if input_text in available_classes_lower:
#             matched_classes.append(available_classes_lower[input_text])
#             continue
        
#         words = input_text.split()
#         for phrase_length in range(4, 1, -1):
#             for i in range(len(words) - phrase_length + 1):
#                 phrase = ' '.join(words[i:i+phrase_length])
#                 if phrase in available_classes_lower:
#                     before_phrase = ' '.join(words[:i])
#                     after_phrase = ' '.join(words[i+phrase_length:])
                    
#                     result = []
#                     if before_phrase:
#                         result.extend(get_valid_classes(before_phrase))
#                     result.append(available_classes_lower[phrase])
#                     if after_phrase:
#                         result.extend(get_valid_classes(after_phrase))
                    
#                     matched_classes.extend(result)
#                     break
#             else:
#                 continue
#             break
#         else:
#             components = []
#             current_word = ""
#             for char in input_text:
#                 if char.isalpha() or char.isdigit():
#                     current_word += char
#                 else:
#                     if current_word:
#                         components.append(current_word)
#                         current_word = ""
#                     if char.strip():
#                         components.append(char)
#             if current_word:
#                 components.append(current_word)
            
#             for component in components:
#                 if not component.strip() or component.lower() in STOP_WORDS:
#                     continue
                    
#                 if component.isdigit():
#                     for digit in component:
#                         if digit.lower() in available_classes_lower:
#                             matched_classes.append(available_classes_lower[digit.lower()])
#                     continue
                    
#                 if len(component) == 1 and component.isalpha():
#                     if component.lower() in available_classes_lower:
#                         matched_classes.append(available_classes_lower[component.lower()])
#                     continue
                    
#                 if len(component) > 1 and component.isalpha():
#                     lower_component = component.lower()
                    
#                     if lower_component in available_classes_lower:
#                         matched_classes.append(available_classes_lower[lower_component])
#                         continue
                        
#                     doc = nlp(lower_component)
#                     lemma = doc[0].lemma_ if doc else lower_component
                    
#                     if lemma in available_classes_lower:
#                         matched_classes.append(available_classes_lower[lemma])
#                         continue
                        
#                     found_variation = False
#                     for root_word, variations in WORD_VARIATIONS_MAP.items():
#                         if lower_component in variations and root_word in available_classes_lower:
#                             matched_classes.append(available_classes_lower[root_word])
#                             found_variation = True
#                             break
                            
#                     if found_variation:
#                         continue
                        
#                     for letter in component.upper():
#                         letter_lower = letter.lower()
#                         if letter_lower in available_classes_lower:
#                             matched_classes.append(available_classes_lower[letter_lower])
#                     continue
                    
#                 lower_component = component.lower()
#                 if lower_component in available_classes_lower:
#                     matched_classes.append(available_classes_lower[lower_component])
#                     continue
                    
#                 doc = nlp(component.lower())
#                 lemmatized_words = [token.lemma_ for token in doc if token.text not in STOP_WORDS]
                
#                 for word in lemmatized_words:
#                     if word in available_classes_lower:
#                         matched_classes.append(available_classes_lower[word])
#                         continue
                        
#                     for root_word, variations in WORD_VARIATIONS_MAP.items():
#                         if word in variations and root_word in available_classes_lower:
#                             matched_classes.append(available_classes_lower[root_word])
#                             continue
                            
#                     closest_match = difflib.get_close_matches(word, available_classes_lower.keys(), n=1, cutoff=0.6)
#                     if closest_match:
#                         matched_classes.append(available_classes_lower[closest_match[0]])
    
#     return matched_classes

# def clear_image_display():
#     """Clear the image display completely"""
#     if hasattr(root, 'word_image_display'):
#         root.word_image_display.config(image=None)
#         root.word_image_label.config(text="")

# def display_image_for_word(word):
#     """Display image only if it exists in the dataset"""
#     # First clear any existing image
#     clear_image_display()
    
#     if not os.path.exists(IMAGE_DATASET_PATH):
#         return False
    
#     clean_word = word.strip().lower()
#     clean_word = ''.join([c for c in clean_word if c.isalpha() or c.isdigit()])
    
#     if not clean_word:
#         return False
    
#     # Only show images for single words
#     if len(clean_word.split()) > 1:
#         return False
    
#     image_extensions = ['.jpg', '.jpeg', '.png', '.gif']
#     image_path = None
    
#     for ext in image_extensions:
#         potential_path = os.path.join(IMAGE_DATASET_PATH, f"{clean_word}{ext}")
#         if os.path.exists(potential_path):
#             image_path = potential_path
#             break
    
#     if not image_path:
#         return False
    
#     try:
#         img = Image.open(image_path)
#         img = img.resize((300, 300), Image.LANCZOS)
        
#         if not hasattr(root, 'word_image_label'):
#             root.word_image_label = ttk.Label(image_frame, text="Image:", style="TLabel")
#             root.word_image_label.pack(pady=(0, 5))
            
#             root.word_image_display = ttk.Label(image_frame)
#             root.word_image_display.pack(pady=(0, 10))
        
#         imgtk = ImageTk.PhotoImage(image=img)
#         root.word_image_display.imgtk = imgtk
#         root.word_image_display.config(image=imgtk)
#         root.word_image_label.config(text=f"Image for '{clean_word}':")
#         return True
#     except Exception as e:
#         print(f"Error loading image: {e}")
#         clear_image_display()
#         return False

# def draw_colored_landmarks(image, results):
#     image_height, image_width, _ = image.shape
    
#     if results.pose_landmarks:
#         POSE_CONNECTIONS_TO_KEEP = [
#             (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_SHOULDER),
#             (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.LEFT_ELBOW),
#             (mp_holistic.PoseLandmark.LEFT_ELBOW, mp_holistic.PoseLandmark.LEFT_WRIST),
#             (mp_holistic.PoseLandmark.RIGHT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_ELBOW),
#             (mp_holistic.PoseLandmark.RIGHT_ELBOW, mp_holistic.PoseLandmark.RIGHT_WRIST),
#             (mp_holistic.PoseLandmark.LEFT_SHOULDER, mp_holistic.PoseLandmark.LEFT_HIP),
#             (mp_holistic.PoseLandmark.RIGHT_SHOULDER, mp_holistic.PoseLandmark.RIGHT_HIP),
#             (mp_holistic.PoseLandmark.LEFT_HIP, mp_holistic.PoseLandmark.RIGHT_HIP),
#             (mp_holistic.PoseLandmark.LEFT_HIP, mp_holistic.PoseLandmark.LEFT_KNEE),
#             (mp_holistic.PoseLandmark.LEFT_KNEE, mp_holistic.PoseLandmark.LEFT_ANKLE),
#             (mp_holistic.PoseLandmark.RIGHT_HIP, mp_holistic.PoseLandmark.RIGHT_KNEE),
#             (mp_holistic.PoseLandmark.RIGHT_KNEE, mp_holistic.PoseLandmark.RIGHT_ANKLE)
#         ]
        
#         for connection in POSE_CONNECTIONS_TO_KEEP:
#             start_idx = connection[0].value
#             end_idx = connection[1].value
            
#             start_point = results.pose_landmarks.landmark[start_idx]
#             end_point = results.pose_landmarks.landmark[end_idx]
            
#             start_x = int(start_point.x * image_width)
#             start_y = int(start_point.y * image_height)
#             end_x = int(end_point.x * image_width)
#             end_y = int(end_point.y * image_height)
            
#             cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["pose_color"], 10)
        
#         if results.face_landmarks:
#             left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER.value]
#             right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER.value]
#             chin = results.face_landmarks.landmark[152]
            
#             neck_x = int(((left_shoulder.x + right_shoulder.x) / 2) * image_width)
#             neck_y = int(((left_shoulder.y + right_shoulder.y) / 2) * image_height)
            
#             chin_x = int(chin.x * image_width)
#             chin_y = int(chin.y * image_height)
            
#             cv2.line(image, (neck_x, neck_y), (chin_x, chin_y), COLORS["neck_color"], 10)
    
#     if results.left_hand_landmarks:
#         for connection in mp_holistic.HAND_CONNECTIONS:
#             start_idx = connection[0]
#             end_idx = connection[1]
            
#             start_point = results.left_hand_landmarks.landmark[start_idx]
#             end_point = results.left_hand_landmarks.landmark[end_idx]
            
#             start_x = int(start_point.x * image_width)
#             start_y = int(start_point.y * image_height)
#             end_x = int(end_point.x * image_width)
#             end_y = int(end_point.y * image_height)
            
#             cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["left_hand_color"], 4)
    
#     if results.right_hand_landmarks:
#         for connection in mp_holistic.HAND_CONNECTIONS:
#             start_idx = connection[0]
#             end_idx = connection[1]
            
#             start_point = results.right_hand_landmarks.landmark[start_idx]
#             end_point = results.right_hand_landmarks.landmark[end_idx]
            
#             start_x = int(start_point.x * image_width)
#             start_y = int(start_point.y * image_height)
#             end_x = int(end_point.x * image_width)
#             end_y = int(end_point.y * image_height)
            
#             cv2.line(image, (start_x, start_y), (end_x, end_y), COLORS["right_hand_color"], 4)
    
#     if results.face_landmarks:
#         outline_connections = [
#             (10, 338), (338, 297), (297, 332), (332, 284), (284, 251), 
#             (251, 389), (389, 356), (356, 454), (454, 323), (323, 361), 
#             (361, 288), (288, 397), (397, 365), (365, 379), (379, 378), 
#             (378, 400), (400, 377), (377, 152), (152, 148), (148, 176), 
#             (176, 149), (149, 150), (150, 136), (136, 172), (172, 58), 
#             (58, 132), (132, 93), (93, 234), (234, 127), (127, 162), 
#             (162, 21), (21, 54), (54, 103), (103, 67), (67, 109), (109, 10)
#         ]
        
#         left_eye_connections = [
#             (33, 7), (7, 163), (163, 144), (144, 145), (145, 153), 
#             (153, 154), (154, 155), (155, 133), (133, 173), (173, 157), 
#             (157, 158), (158, 159), (159, 160), (160, 161), (161, 246), 
#             (246, 33)
#         ]
        
#         right_eye_connections = [
#             (263, 249), (249, 390), (390, 373), (373, 374), (374, 380), 
#             (380, 381), (381, 382), (382, 362), (362, 398), (398, 384), 
#             (384, 385), (385, 386), (386, 387), (387, 388), (388, 466), 
#             (466, 263)
#         ]
        
#         lips_outer_connections = [
#             (61, 146), (146, 91), (91, 181), (181, 84), (84, 17), 
#             (17, 314), (314, 405), (405, 321), (321, 375), (375, 291), 
#             (291, 61)
#         ]
        
#         lips_inner_connections = [
#             (78, 95), (95, 88), (88, 178), (178, 87), (87, 14), 
#             (14, 317), (317, 402), (402, 318), (318, 324), (324, 308), 
#             (308, 78)
#         ]
        
#         nose_line = [
#             (1, 168)
#         ]
        
#         for connection_set in [outline_connections, left_eye_connections, 
#                               right_eye_connections, lips_outer_connections, 
#                               lips_inner_connections, nose_line]:
#             for connection in connection_set:
#                 start_idx = connection[0]
#                 end_idx = connection[1]
                
#                 start_point = results.face_landmarks.landmark[start_idx]
#                 end_point = results.face_landmarks.landmark[end_idx]
                
#                 start_x = int(start_point.x * image_width)
#                 start_y = int(start_point.y * image_height)
#                 end_x = int(end_point.x * image_width)
#                 end_y = int(end_point.y * image_height)
                
#                 line_color = COLORS["face_color"]
#                 line_thickness = 3
                
#                 if connection_set is nose_line:
#                     line_color = COLORS["face_color"]
#                     line_thickness = 5
                
#                 cv2.line(image, (start_x, start_y), (end_x, end_y), line_color, line_thickness)
    
#     return image

# def visualize_colored_landmarks(class_names, video_label=None):
#     if not video_label:
#         video_label = globals().get('video_label', None)
#         if not video_label:
#             return
    
#     # Clear ALL displays before processing new input
#     clear_image_display()
#     video_label.config(image=None)
    
#     if not class_names:
#         status_label.config(text="No valid signs found", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
#         return
    
#     status_label.config(text="Processing...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
#     root.update()
    
#     # Process each sign one by one
#     for class_name in class_names:
#         # Clear image display before each new word
#         clear_image_display()
        
#         # Try to display image for this word and check if successful
#         image_found = display_image_for_word(class_name)
        
#         # If no image was found, update the label to indicate that
#         if not image_found:
#             root.word_image_label.config(text=f"No image available for '{class_name}'")
        
#         # Process sign language video
#         folder_path = os.path.join(DATASET_PATH, class_name)
#         if not os.path.exists(folder_path):
#             continue
        
#         video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
#         if not video_files:
#             continue

#         video_path = os.path.join(folder_path, video_files[0])
#         cap = cv2.VideoCapture(video_path)

#         with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#             while cap.isOpened():
#                 ret, frame = cap.read()
#                 if not ret:
#                     break

#                 frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#                 results = holistic.process(frame)
                
#                 black_frame = np.zeros(frame.shape, dtype=np.uint8)
#                 black_frame = draw_colored_landmarks(black_frame, results)
                
#                 frame_resized = cv2.resize(black_frame, (video_label.winfo_width(), video_label.winfo_height()))
#                 img = Image.fromarray(frame_resized)
#                 imgtk = ImageTk.PhotoImage(image=img)

#                 video_label.imgtk = imgtk
#                 video_label.configure(image=imgtk)
#                 video_label.update()

#                 if cv2.waitKey(10) & 0xFF == ord('q'):
#                     break

#         cap.release()
    
#     if class_names:
#         status_label.config(text="Translation Complete", font=("Poppins", 14, "bold"), foreground=COLORS["secondary"])
#     else:
#         status_label.config(text="No valid signs found", font=("Poppins", 14, "bold"), foreground=COLORS["error"])

# def on_visualize(event=None):
#     input_text = class_entry.get("1.0", tk.END).strip()
    
#     # Clear ALL displays before processing new input
#     clear_image_display()
#     if hasattr(root, 'video_label'):
#         root.video_label.config(image=None)
    
#     valid_classes = get_valid_classes(input_text)

#     if valid_classes:
#         status_label.config(text="Processing...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
#         root.update()
#         visualize_colored_landmarks(valid_classes)
#     else:
#         status_label.config(text="No valid sign found", font=("Poppins", 14, "bold"), foreground=COLORS["error"])
#         clear_image_display()
        
# def listen_to_speech():
#     recognizer = sr.Recognizer()
    
#     try:
#         with sr.Microphone() as source:
#             mic_button.config(text="🎤 Listening...", style="Listening.TButton")
#             status_label.config(text="Listening... Speak now", font=("Poppins", 18, "bold"), foreground=COLORS["secondary"])
#             root.update()
            
#             recognizer.adjust_for_ambient_noise(source, duration=1)
            
#             try:
#                 audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
#                 try:
#                     text = recognizer.recognize_google(audio)
#                     speech_queue.put(("success", text))
                    
#                     class_entry.delete("1.0", tk.END)
#                     class_entry.insert("1.0", text)
                    
#                     # Clear displays before processing new input
#                     clear_image_display()
#                     if hasattr(root, 'video_label'):
#                         root.video_label.config(image=None)
                    
#                     valid_classes = get_valid_classes(text)
#                     if valid_classes:
#                         status_label.config(text="Processing speech...", font=("Poppins", 14, "bold"), foreground=COLORS["text_primary"])
#                         root.update()
#                         visualize_colored_landmarks(valid_classes)
#                     else:
#                         status_label.config(text="No valid sign found in speech", font=("Poppins", 14, "bold"), foreground=COLORS["error"])
#                         clear_image_display()
                        
#                 except sr.UnknownValueError:
#                     speech_queue.put(("error", "Could not understand audio"))
#                 except sr.RequestError as e:
#                     speech_queue.put(("error", f"Speech service error: {e}"))
#             except sr.WaitTimeoutError:
#                 speech_queue.put(("error", "Listening timed out"))
#     except OSError as e:
#         speech_queue.put(("error", f"Microphone error: {e}"))
#     finally:
#         speech_queue.put(("reset_button", ""))

# def start_speech_recognition():
#     mic_button.config(state=tk.DISABLED)
#     threading.Thread(target=listen_to_speech, daemon=True).start()
#     root.after(100, check_speech_queue)

# def check_speech_queue():
#     try:
#         while not speech_queue.empty():
#             result_type, content = speech_queue.get_nowait()
            
#             if result_type == "success":
#                 status_label.config(text="Speech translated successfully", font=("Poppins", 14, "bold"), foreground=COLORS["secondary"])
#             elif result_type == "error":
#                 status_label.config(text=content, font=("Poppins", 14, "bold"), foreground=COLORS["error"])
#             elif result_type == "reset_button":
#                 mic_button.config(text="🎤 Speak", style="TButton", state=tk.NORMAL)
#     except:
#         pass
    
#     root.after(100, check_speech_queue)

# # GUI Setup
# root = tk.Tk()
# root.title("Sign Language Translator")
# root.geometry("1400x900")
# root.configure(bg=COLORS["dark_bg"])

# # Custom Fonts
# title_font = tkFont.Font(family="Poppins", size=24, weight="bold")
# label_font = tkFont.Font(family="Poppins", size=14, weight="bold")
# button_font = tkFont.Font(family="Poppins", size=14, weight="bold")
# status_font = tkFont.Font(family="Poppins", size=12)
# entry_font = tkFont.Font(family="Poppins", size=12)

# # Custom Styles
# style = ttk.Style(root)
# style.theme_use("clam")

# style.configure("TFrame", background=COLORS["light_bg"])
# style.configure("Title.TFrame", background=COLORS["dark_bg"])
# style.configure("TLabel", background=COLORS["light_bg"], foreground=COLORS["text_primary"], font=label_font)
# style.configure("Title.TLabel", background=COLORS["dark_bg"], foreground=COLORS["primary"], font=title_font)
# style.configure("TButton", background=COLORS["primary"], foreground=COLORS["dark_bg"], 
#                 font=button_font, borderwidth=0, focusthickness=3, focuscolor=COLORS["primary"])
# style.map("TButton", 
#           background=[("active", COLORS["primary_variant"])],
#           foreground=[("active", COLORS["dark_bg"])])
# style.configure("Listening.TButton", background=COLORS["error"], foreground=COLORS["text_primary"])

# # Title Bar
# title_bar = ttk.Frame(root, style="Title.TFrame")
# title_bar.pack(fill=tk.X, side=tk.TOP, pady=(10, 0))

# # title_label = ttk.Label(title_bar, text="Sign Language Translator", style="Title.TLabel")
# # title_label.pack(pady=(10, 20))

# # Main Frame
# main_frame = ttk.Frame(root, padding="0")
# main_frame.pack(fill=tk.BOTH, expand=True)

# # Layout Frame
# layout_frame = ttk.Frame(main_frame)
# layout_frame.pack(fill=tk.BOTH, expand=True)

# # Left Section (Input + Image)
# left_frame = ttk.Frame(layout_frame, padding="20", style="TFrame")
# left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)

# # Input Section
# input_frame = ttk.Frame(left_frame, style="TFrame")
# input_frame.pack(fill=tk.BOTH, expand=True)

# class_label = ttk.Label(input_frame, text="Enter Text:", style="TLabel")
# class_label.pack(pady=(0, 10))

# class_entry = tk.Text(input_frame, font=("Poppins", 16, "bold"), height=14, width=30, wrap="word", 
#                      bg=COLORS["lighter_bg"], fg=COLORS["text_primary"], 
#                      insertbackground=COLORS["text_primary"],
#                      highlightthickness=1, highlightbackground=COLORS["primary"],
#                      highlightcolor=COLORS["primary"], relief="flat")
# class_entry.pack(pady=10, ipady=10, fill=tk.BOTH, expand=True)

# class_entry.bind("<Return>", on_visualize)

# button_frame = ttk.Frame(input_frame)
# button_frame.pack(pady=10)

# visualize_button = ttk.Button(button_frame, text="Translate", command=on_visualize, style="TButton")
# visualize_button.pack(side=tk.LEFT, padx=5)

# mic_button = ttk.Button(button_frame, text="🎤 Speak", command=start_speech_recognition, style="TButton")
# mic_button.pack(side=tk.LEFT, padx=5)

# status_label = ttk.Label(input_frame, text="", style="TLabel", anchor="w")
# status_label.pack(pady=10)

# # Image display frame (below input section)
# image_frame = ttk.Frame(left_frame, height=300, style="TFrame")
# image_frame.pack(fill=tk.BOTH, expand=False)

# # Initialize image display labels
# root.word_image_label = ttk.Label(image_frame, text="", style="TLabel")
# root.word_image_label.pack(pady=(0, 5))
# root.word_image_display = ttk.Label(image_frame)
# root.word_image_display.pack(pady=(0, 10))

# # Right Section (Video Output)
# output_frame = ttk.Frame(layout_frame, padding="10", style="TFrame")
# output_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)

# video_label = ttk.Label(output_frame, text="Sign Language Output", style="TLabel")
# video_label.pack(fill=tk.BOTH, expand=True)

# # Run the application
# root.mainloop()